In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product
import gurobipy as gb
from gurobipy import GRB

repeats = json.load(open('../cleanerdata/repeats.json'))
repeats = {int(k):v for k,v in repeats.items()}

locdem = pd.read_excel('../cleanerdata/locdem.xlsx')

q = locdem['Number of pallets'].to_numpy().astype(float).tolist()
q = [0] + q
q.extend([v['dem'] for v in repeats.values()])

locs = pd.read_excel('../cleanerdata/locations.xlsx')
longs, lats = locs['long'].to_numpy().tolist(), locs['lat'].to_numpy().tolist()
longs.extend([longs[v['map']] for v in repeats.values()])
lats.extend([lats[v['map']] for v in repeats.values()])

distmat = pd.read_json('../cleanerdata/distmat.json').to_numpy()
timemat = pd.read_json('../cleanerdata/timemat.json').to_numpy()
def get(i):
    try:
        i = repeats[i]['map']
    except KeyError:
        pass
    return i

def cost(i,j):
    i,j = get(i), get(j)
    return distmat[i,j]
def time(i,j):
    i,j = get(i), get(j)
    return timemat[i,j]

def location(i):
    return longs[i], lats[i]

In [2]:
loops = json.load(open('./loops.json'))
loops

[[0, 46, 61, 60, 3, 0],
 [0, 51, 105, 24, 82, 172, 125, 0],
 [0, 68, 49, 1, 45, 0],
 [0, 124, 67, 166, 85, 111, 34, 0],
 [0, 2, 86, 89, 132, 123, 0],
 [0, 28, 95, 103, 19, 119, 40, 0],
 [0, 88, 66, 12, 0],
 [0, 115, 26, 151, 87, 178, 22, 0],
 [0, 43, 134, 168, 120, 4, 106, 0],
 [0, 160, 78, 131, 135, 0],
 [0, 161, 167, 69, 152, 113, 32, 0],
 [0, 162, 9, 133, 107, 0],
 [0, 25, 99, 13, 74, 20, 0],
 [0, 53, 29, 54, 127, 0],
 [0, 64, 65, 70, 126, 0],
 [0, 90, 80, 76, 23, 100, 83, 5, 0],
 [0, 11, 110, 137, 6, 0],
 [0, 75, 31, 136, 84, 130, 0],
 [0, 153, 108, 129, 35, 138, 0],
 [0, 171, 81, 93, 33, 128, 169, 0],
 [0, 15, 170, 173, 117, 50, 0],
 [0, 47, 44, 101, 102, 0],
 [0, 59, 0],
 [0, 63, 7, 16, 48, 0],
 [0, 176, 79, 109, 18, 112, 58, 0],
 [0, 10, 163, 72, 164, 0],
 [0, 21, 122, 165, 17, 56, 57, 0],
 [0, 62, 180, 55, 104, 92, 0],
 [0, 97, 71, 98, 8, 27, 0],
 [0, 30, 148, 42, 14, 177, 77, 0],
 [0, 39, 121, 91, 157, 96, 0],
 [0, 149, 158, 118, 73, 0],
 [0, 159, 94, 52, 179, 150, 0],
 [0, 38

In [8]:
h = np.zeros(len(loops), dtype=float)
for i, loop in enumerate(loops):
    for j in range(0, len(loop)-1):
        arc = loop[j], loop[j+1]
        h[i] += time(*arc)
h = h / 3600

In [9]:
D = range(16)
L = range(0, len(loops))
T = range(9)

m = gb.Model()
z = m.addVars(D, vtype=GRB.BINARY, name='z')
y = m.addVars(L,T,D,vtype=GRB.BINARY, name='y')

m.setObjective(z.sum('*'), GRB.MINIMIZE)
m.addConstrs((y.sum(l,'*', '*') == 1 for l in L))
m.addConstrs((
    sum([h[l]*y[l,t,d] for l in L]) <= 8 for t in T for d in D
))
m.addConstrs((y[l,t,d] <= z[d] for l in L for t in T for d in D))
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5509 rows, 5344 columns and 21312 nonzeros
Model fingerprint: 0x65084e95
Variable types: 0 continuous, 5344 integer (5344 binary)
Coefficient statistics:
  Matrix range     [3e-01, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 15.0000000
Presolve time: 0.03s
Presolved: 5509 rows, 5344 columns, 21312 nonzeros
Variable types: 0 continuous, 5344 integer (5344 binary)

Root relaxation: objective 1.111111e-01, 5342 iterations, 0.06 seconds (0.10 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.11111    0  334   15.00000    0.11111  99.3%     -    0s
H    0     0                      14.0000000    0.1111

In [10]:
y

{(0, 0, 0): <gurobi.Var y[0,0,0] (value -0.0)>,
 (0, 0, 1): <gurobi.Var y[0,0,1] (value -0.0)>,
 (0, 0, 2): <gurobi.Var y[0,0,2] (value -0.0)>,
 (0, 0, 3): <gurobi.Var y[0,0,3] (value -0.0)>,
 (0, 0, 4): <gurobi.Var y[0,0,4] (value -0.0)>,
 (0, 0, 5): <gurobi.Var y[0,0,5] (value -0.0)>,
 (0, 0, 6): <gurobi.Var y[0,0,6] (value -0.0)>,
 (0, 0, 7): <gurobi.Var y[0,0,7] (value -0.0)>,
 (0, 0, 8): <gurobi.Var y[0,0,8] (value -0.0)>,
 (0, 0, 9): <gurobi.Var y[0,0,9] (value -0.0)>,
 (0, 0, 10): <gurobi.Var y[0,0,10] (value -0.0)>,
 (0, 0, 11): <gurobi.Var y[0,0,11] (value -0.0)>,
 (0, 0, 12): <gurobi.Var y[0,0,12] (value -0.0)>,
 (0, 0, 13): <gurobi.Var y[0,0,13] (value -0.0)>,
 (0, 0, 14): <gurobi.Var y[0,0,14] (value -0.0)>,
 (0, 0, 15): <gurobi.Var y[0,0,15] (value 0.0)>,
 (0, 1, 0): <gurobi.Var y[0,1,0] (value -0.0)>,
 (0, 1, 1): <gurobi.Var y[0,1,1] (value -0.0)>,
 (0, 1, 2): <gurobi.Var y[0,1,2] (value -0.0)>,
 (0, 1, 3): <gurobi.Var y[0,1,3] (value -0.0)>,
 (0, 1, 4): <gurobi.Var y[0,1

In [11]:
Loops = []
for loop in loops:
    L = []
    for l in loop:
        if l in repeats:
            L.append(repeats[l]['map'])
        else:
            L.append(l)
    Loops.append(L)

In [12]:
Loops

[[0, 46, 61, 60, 3, 0],
 [0, 51, 105, 24, 82, 111, 5, 0],
 [0, 68, 49, 1, 45, 0],
 [0, 124, 67, 105, 85, 111, 34, 0],
 [0, 2, 86, 89, 12, 123, 0],
 [0, 28, 95, 103, 19, 119, 40, 0],
 [0, 88, 66, 12, 0],
 [0, 115, 26, 40, 87, 119, 22, 0],
 [0, 43, 32, 107, 120, 4, 106, 0],
 [0, 43, 78, 9, 32, 0],
 [0, 43, 106, 69, 40, 113, 32, 0],
 [0, 43, 9, 32, 107, 0],
 [0, 25, 99, 13, 74, 20, 0],
 [0, 53, 29, 54, 5, 0],
 [0, 64, 65, 70, 5, 0],
 [0, 90, 80, 76, 23, 100, 83, 5, 0],
 [0, 11, 110, 33, 6, 0],
 [0, 75, 31, 33, 84, 6, 0],
 [0, 40, 108, 6, 35, 33, 0],
 [0, 110, 81, 93, 33, 6, 108, 0],
 [0, 15, 109, 112, 117, 50, 0],
 [0, 47, 44, 101, 102, 0],
 [0, 59, 0],
 [0, 63, 7, 16, 48, 0],
 [0, 117, 79, 109, 18, 112, 58, 0],
 [0, 10, 97, 72, 98, 0],
 [0, 21, 122, 104, 17, 56, 57, 0],
 [0, 62, 122, 55, 104, 92, 0],
 [0, 97, 71, 98, 8, 27, 0],
 [0, 30, 39, 42, 14, 118, 77, 0],
 [0, 39, 121, 91, 42, 96, 0],
 [0, 39, 42, 118, 73, 0],
 [0, 42, 94, 52, 121, 39, 0],
 [0, 38, 41, 37, 36, 114, 0],
 [0, 38, 37,